# BRIDGE-ML Quickstart Guide

This notebook demonstrates the basic functionality of BRIDGE-ML (Bi-directional Reasoning for Intelligence Data Graph Evolution Using Machine Learning).

First, ensure you have the correct conda environment activated:
```bash
conda activate bridge-ml
```

In [ ]:
# Import necessary libraries
import torch
import networkx as nx
import matplotlib.pyplot as plt
from bridge_ml.core.matcher import OntologyMatcher
from bridge_ml.utils.graph import build_graph
from bridge_ml.utils.visualization import plot_ontology_graph

## 1. Creating Sample Ontologies

Let's create two simple example ontologies to demonstrate the matching process.

In [ ]:
# Create sample source ontology
source_ontology = {
    'concepts': {
        'Person': {
            'properties': ['name', 'age', 'address'],
            'relations': ['worksAt', 'livesIn']
        },
        'Organization': {
            'properties': ['name', 'location'],
            'relations': ['hasEmployee']
        },
        'Location': {
            'properties': ['name', 'coordinates'],
            'relations': ['containsOrg', 'hasResident']
        }
    }
}

# Create sample target ontology
target_ontology = {
    'concepts': {
        'Individual': {
            'properties': ['fullName', 'age', 'residence'],
            'relations': ['employedBy', 'residentOf']
        },
        'Company': {
            'properties': ['companyName', 'address'],
            'relations': ['employs']
        },
        'Place': {
            'properties': ['placeName', 'geoCoordinates'],
            'relations': ['hostsBusiness', 'houses']
        }
    }
}

## 2. Visualizing the Ontologies

Let's visualize both ontologies to better understand their structure.

In [ ]:
def create_networkx_graph(ontology):
    G = nx.Graph()
    
    # Add nodes for concepts
    for concept, details in ontology['concepts'].items():
        G.add_node(concept, node_type='concept')
        
        # Add property nodes
        for prop in details['properties']:
            prop_node = f"{concept}_{prop}"
            G.add_node(prop_node, node_type='property')
            G.add_edge(concept, prop_node)
    
        # Add relation edges
        for rel in details['relations']:
            G.add_node(rel, node_type='relation')
            G.add_edge(concept, rel)
    
    return G

def plot_graph(G, title):
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos,
                          node_color='lightblue',
                          node_size=2000,
                          nodelist=[n for n, d in G.nodes(data=True) if d['node_type'] == 'concept'])
    nx.draw_networkx_nodes(G, pos,
                          node_color='lightgreen',
                          node_size=1500,
                          nodelist=[n for n, d in G.nodes(data=True) if d['node_type'] == 'property'])
    nx.draw_networkx_nodes(G, pos,
                          node_color='lightpink',
                          node_size=1500,
                          nodelist=[n for n, d in G.nodes(data=True) if d['node_type'] == 'relation'])
    
    # Draw edges
    nx.draw_networkx_edges(G, pos)
    
    # Add labels
    nx.draw_networkx_labels(G, pos)
    
    plt.title(title)
    plt.axis('off')
    plt.show()

# Visualize both ontologies
G_source = create_networkx_graph(source_ontology)
G_target = create_networkx_graph(target_ontology)

plot_graph(G_source, 'Source Ontology')
plot_graph(G_target, 'Target Ontology')

## 3. Performing Ontology Matching

Now let's use BRIDGE-ML to perform ontology matching between these two ontologies.

In [ ]:
# Initialize the matcher
matcher = OntologyMatcher()

# Perform matching
try:
    alignments = matcher.match(source_ontology, target_ontology)
    print("Alignments found:")
    print(alignments)
except NotImplementedError:
    print("Note: This is a placeholder. Implementation of the match method is required.")
    # For demonstration, let's show what the alignments might look like
    example_alignments = {
        'concept_matches': [
            ('Person', 'Individual', 0.95),
            ('Organization', 'Company', 0.88),
            ('Location', 'Place', 0.92)
        ],
        'property_matches': [
            ('name', 'fullName', 0.85),
            ('age', 'age', 1.0),
            ('address', 'residence', 0.78)
        ],
        'relation_matches': [
            ('worksAt', 'employedBy', 0.82),
            ('livesIn', 'residentOf', 0.88),
            ('hasEmployee', 'employs', 0.90)
        ]
    }
    print("\nExample of expected alignments:")
    for category, matches in example_alignments.items():
        print(f"\n{category.replace('_', ' ').title()}:")
        for source, target, confidence in matches:
            print(f"{source:15} → {target:15} (confidence: {confidence:.2f})")

## 4. Visualizing Alignments

Let's create a visualization of the alignments between the two ontologies.

In [ ]:
def plot_alignments(source_ontology, target_ontology, alignments):
    plt.figure(figsize=(15, 8))
    
    # Create positions for nodes
    source_concepts = list(source_ontology['concepts'].keys())
    target_concepts = list(target_ontology['concepts'].keys())
    
    # Position nodes
    pos = {}
    for i, concept in enumerate(source_concepts):
        pos[f"source_{concept}"] = (-2, (len(source_concepts)-1)/2 - i)
    for i, concept in enumerate(target_concepts):
        pos[f"target_{concept}"] = (2, (len(target_concepts)-1)/2 - i)
    
    # Create graph
    G = nx.Graph()
    
    # Add nodes
    for concept in source_concepts:
        G.add_node(f"source_{concept}", side='source')
    for concept in target_concepts:
        G.add_node(f"target_{concept}", side='target')
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos,
                          nodelist=[n for n, d in G.nodes(data=True) if d['side'] == 'source'],
                          node_color='lightblue',
                          node_size=2000)
    nx.draw_networkx_nodes(G, pos,
                          nodelist=[n for n, d in G.nodes(data=True) if d['side'] == 'target'],
                          node_color='lightgreen',
                          node_size=2000)
    
    # Add labels
    labels = {}
    for concept in source_concepts:
        labels[f"source_{concept}"] = concept
    for concept in target_concepts:
        labels[f"target_{concept}"] = concept
    nx.draw_networkx_labels(G, pos, labels)
    
    # Draw alignments
    for source, target, confidence in alignments['concept_matches']:
        plt.plot([pos[f"source_{source}"][0], pos[f"target_{target}"][0]],
                 [pos[f"source_{source}"][1], pos[f"target_{target}"][1]],
                 'gray', alpha=confidence, linewidth=confidence*3)
    
    plt.title('Ontology Alignments')
    plt.axis('off')
    plt.show()

# Plot the alignments
plot_alignments(source_ontology, target_ontology, example_alignments)

## 5. Next Steps

This quickstart guide demonstrated the basic usage of BRIDGE-ML. Next steps could include:

1. Implementing custom matching algorithms
2. Adding more sophisticated visualization techniques
3. Integrating with existing ontology formats (OWL, RDF)
4. Adding evaluation metrics for alignment quality
5. Implementing machine learning models for improved matching